This script opens ICESat-02 ATL08 v6 data to export photons

Ideally, file structure should be as follows:
- The current working directory contains this script, and separate subdirectories for raw data and for extracted data
- This script will be the middleman, pulling data and writing data between these two subdirectories

                                                 Current Working Directory
___________________________________________________________________________________________________________________________
                 ↓                                     ↓                                            ↓
            ./rawData/    <------->     height_extraction.ipynb (this script)    <------->  ./extractedData/

Setting our current working directory to proper location, and subdirectories for raw & extracted data

In [1]:
import os
os.getcwd()
os.chdir(r'D:/Icesat')
os.getcwd()
os.listdir()
rawData = './rawData'
rawDataTesting = './rawDataTesting'
rawDataTestingLarger = './rawDataTestingLarger'
extractedData = './extractedData'

Importing necessary packages: h5py, numpy, etc

In [2]:
import h5py
import numpy as np
from numpy import savetxt
import pandas as pd
import shapely as spl
from datetime import datetime

Dr. Thomas's instructions: "Extract relevant data from h5 file (h_canopy, h_canopy_uncertainty, night_flag, lat, long, and file name (because the beam and date is contained in the name) into text or whatever (we already have some code to do this).   You will want a file with the following information from icesat: Shotnumber, date, beam (or ground track), lat, lon, h_canopy and associated metrics, 20m segment heights, night_flag, h_canopy_uncertainty."

High level overview of code: 
For every file in our 5000 file dataset, and for every beam of the 6 beams within that specific file, we'll assign variables to parameters of interest (lat, long, night_flag) and write those values to csv 

Outer loop: current file in subdirectory
Inner loop: beams of current file
Innermost code: assigning, reshaping, and getting all parameters into a single table that we'll write to csv 


In [3]:
# First creating a row for column names
# Doing this before any files are read as we only want 1 row of column names,
# rather than repeating column names every time a new file is active
column_names = np.array((['file name',
                'date',
                'beam name',
                'latitude',
                'longitude',
                'spacecraft orientation',
                'canopy height',
                'canopy height uncertainty',
                'canopy height metric 1',
                'canopy height metric 2', 
                'canopy height metric 3', 
                'canopy height metric 4', 
                'canopy height metric 5', 
                'canopy height metric 6', 
                'canopy height metric 7', 
                'canopy height metric 8', 
                'canopy height metric 9', 
                'canopy height metric 10', 
                'canopy height metric 11', 
                'canopy height metric 12', 
                'canopy height metric 13', 
                'canopy height metric 14', 
                'canopy height metric 15', 
                'canopy height metric 16', 
                'canopy height metric 17',
                'canopy height metric 18', 
                'canopy height 0-20m',
                'canopy height 20-40m',
                'canopy height 40-60m',
                'canopy height 60-80m',
                'canopy height 80-100m',
                'canopy openness',
                'segment id beginning',
                'segment id ending',
                'night flag']))

num = 0
for name in column_names:
    num = num + 1
column_names = pd.DataFrame(column_names.reshape((1, 35)))
print('Dimensions of column names',column_names.shape)

Dimensions of column names (1, 35)


In [58]:
def getBeams(file):
    # Getting all the beams for a file
    allKeys = list(file.keys())
    
    # number of keys that begin with 'gt', or in otherwords a beam
    beamCount = sum('gt' in key for key in list(file.keys()))
    
    # surfType is the parameter just before the keys begin
    surfTypeIndex = allKeys.index('ds_surf_type')
    
    # slicing the allKeys array from surfType to the index of surfType + number of beams
    beams = np.array(allKeys[surfTypeIndex + 1 : surfTypeIndex + beamCount + 1])
    
    # print('file:', file, 'has the following {} beams'.format(beams.shape[0]), beams)
    
    # Return an array for all the beams only in that specific file (could be 6, could be only 2)
    return beams

In [59]:
# Workhorse function to extract data from a given file
def extractFileData(file, beams, outputFile):
    
    rowsInFile = 0

    # Assigning an array to the beams present in the specific file
    print('file beams:', beams)
    
    # Column for spacecraft orientation
    orientation = file['orbit_info/sc_orient'][0]
        
    # Looping through all the beams for the file
    # Keep in mind that any given beam may or may not have the land segments group
    for beam in beams:
        
        # If there is a land_segments flag, then we want to extract the data and put it into a text file
        if '{}/land_segments'.format(beam) in file:
            print('{} does have a land_segments flag'.format(beam))
            rowsInBeam = writeBeamToFile(file, beam, outputFile)
            
            rowsInFile = rowsInFile + rowsInBeam
        else:
            print('{} DOES NOT have a land_segments flag'.format(beam))
            
        # return df
    print('file', file, 'has', rowsInFile, 'total rows')
    return rowsInFile

In [63]:
# Function to assign H5 dataframes to arrays and write to a file
def writeBeamToFile(file, beam, outputFile):
    latitude = file[beam + '/land_segments/latitude']
    latitude = np.array((latitude)).reshape((latitude.shape[0], 1))
    # print('beam', beam, 'has', latitude.shape[0], 'records in the latitude flag')


    longitude = file[beam + '/land_segments/longitude']
    longitude = np.array((longitude)).reshape((longitude.shape[0], 1))
    # print('beam', beam, 'has', latitude.shape[0], 'records in the longitude flag')
   
    # Column for spacecraft orientation, reassigned in loop to match length of latitude records for particular beam 
    orientation = file['orbit_info/sc_orient'][0]
    sc_orient = np.full_like(longitude, orientation, dtype=int)
    # print(sc_orient[0:2])

    # Column for canopy height             
    h_canopy = file[beam + '/land_segments/canopy/h_canopy']
    h_canopy = np.array(h_canopy).reshape((h_canopy.shape[0], 1))       

    # Column for canopy height uncertainty
    h_canopy_uncertainty = file[beam + '/land_segments/canopy/h_canopy_uncertainty']
    h_canopy_uncertainty = np.array(h_canopy_uncertainty).reshape((h_canopy_uncertainty.shape[0], 1))        

    # This metric is NOT reshaped into a 1 column array, it has dimensions of ( number of transects x 18)
    canopy_h_metrics = file[beam + '/land_segments/canopy/canopy_h_metrics']
    canopy_h_metrics = np.array(canopy_h_metrics)

    # This metric is also NOT reshaped into a 1 column array, it has dimensions of ( number of transects x 5)
    h_canopy_20m = file[beam + '/land_segments/canopy/h_canopy_20m/']
    h_canopy_20m = np.array(h_canopy_20m)

    # Column for the canopy openness
    canopy_openness = file[beam + '/land_segments/canopy/canopy_openness']
    canopy_openness = np.array(canopy_openness).reshape((canopy_openness.shape[0], 1))

    # Column for the segment ID beginning
    segment_id_beg = file[beam + '/land_segments/segment_id_beg']
    segment_id_beg = np.array(segment_id_beg).reshape((segment_id_beg.shape[0], 1))
    # print('Segment ID beginning', segment_id_beg[0:2])

    # Column for the segment ID ending
    segment_id_end = file[beam + '/land_segments/segment_id_end']
    segment_id_end = np.array(segment_id_end).reshape((segment_id_end.shape[0], 1))

    # Column for the night flag
    night_flag = file[beam + '/land_segments/night_flag']
    night_flag = np.array(night_flag).reshape((night_flag.shape[0], 1))

    # Column for the specific file name, same dimensions as latitude column
    # Though this is the first column in the eventual csv, it is lower in this loop as we dont yet know # of records per specific beam. 
    # That information is obtained with the latitude flag at the start of the loop
    file_name = np.full_like(latitude, filename[:-3], dtype=np.dtype('U100'))

    # Creating a column for the granule date.
    # HMS data can be parsed later as there's inconsistencies between file names and the granule START time listed in the Earthdata portal. 
    # Creating a datetime object of the filename string, then a column of just the datetime information
    date_time = datetime.strptime(filename[filename.index('_') + 1:filename.index('_') + 15], "%Y%m%d%H%M%S")
    date = np.full_like(latitude, date_time, dtype=np.dtype('U100'))
    date = np.array(date).reshape((date.shape[0], 1))

    # Creating a column for the specific beam name, same dimensions as the latitude column
    beam_name = np.full_like(file_name, beam, dtype=np.dtype('U100'))            
    
    rowsInBeam = 0
    rowsInBeam = latitude.shape[0]
    print('beam',beam,'has', rowsInBeam, 'records')

    # overall_records = overall_records + records_in_beam
    # print('output csv will have', overall_records, 'records with addition of', beam)

    df = pd.DataFrame(np.hstack((
            file_name,
            date,
            beam_name,            
            latitude, 
            longitude, 
            sc_orient, 
            h_canopy, 
            h_canopy_uncertainty, 
            canopy_h_metrics, 
            h_canopy_20m,
            canopy_openness, 
            segment_id_beg, 
            segment_id_end, 
            night_flag                            
        )))
    df.to_csv(output_file, mode='a', index=False, header=False)
    return rowsInBeam


In [64]:
# def writeBeamToFile(df, output_file):
#     df.to_csv(output_file, mode='a', index=False, header=False)

Code block below is smaller version of main chunk, used just for looping through rawDataTesting

Broad level structure:
- For every file in subdirectory:
    - create file object for current file
    - get beam information for current file:
    - for every beam in current file:
        - create a dataframe of the parameters
        - write the dataframe to a textfile

In [67]:
# Creating an output file and clearing it's contents
outputFile = "./extractedData/output.txt"
open(outputFile, 'w').close()

filesExtracted = 0
totalRows = 0
rowsInFile = 0

# Looping through all files in directory
for filename in sorted(os.listdir(rawDataTestingLarger)):
    file = os.path.join(rawDataTestingLarger, filename)
    print('\ncurrent file:', file)
   
    # Testing that the current file path is valid by opening it with h5py object
    try:
        file = h5py.File(file, 'r')
        print('File open success!')
        
        # If successfully opened, extract the data from file (main code chunk) and write to .txt 
        beams = getBeams(file)
        rowsInFile = extractFileData(file, beams, outputFile)
        # writeBeamToFile(df, output_file)
        
    # If file can't be opened, printing error message and moving on
    except IOError as e:
        print('File open FAILURE')
        print(str(e))
    
    filesExtracted = filesExtracted + 1
    totalRows = totalRows + rowsInFile

print('\ntotal files extracted:', filesExtracted)
print('\ntotal rows in output txt file', totalRows)


current file: ./rawDataTestingLarger\ATL08_20190323011928_12920206_006_02.h5
File open FAILURE
Unable to open file (file signature not found)

current file: ./rawDataTestingLarger\ATL08_20190328005358_13680207_006_02.h5
File open success!
file beams: ['gt1l' 'gt1r']
gt1l does have a land_segments flag
beam gt1l has 50 records
gt1r DOES NOT have a land_segments flag
file <HDF5 file "ATL08_20190328005358_13680207_006_02.h5" (mode r)> has 50 total rows

current file: ./rawDataTestingLarger\ATL08_20190715084425_02640402_006_02.h5
File open success!
file beams: ['gt1l' 'gt1r' 'gt2l' 'gt2r' 'gt3l' 'gt3r']
gt1l does have a land_segments flag
beam gt1l has 132 records
gt1r does have a land_segments flag
beam gt1r has 93 records
gt2l does have a land_segments flag
beam gt2l has 97 records
gt2r does have a land_segments flag
beam gt2r has 68 records
gt3l does have a land_segments flag
beam gt3l has 75 records
gt3r does have a land_segments flag
beam gt3r has 44 records
file <HDF5 file "ATL08_20

This is the old code chunk for looping through the files, leave it untouched and use it to copy chunks of code for more elegant looping with function calls

In [18]:
# Creating an output file and clearing it's contents
output_file = "./extractedData/output.txt"
open(output_file, 'w').close()

# Test variable to track number of measurements in current beam
# Test variable for running tally of overall records 
# records_in_beam = 0
# overall_records = 0

# Adding our previous column names array to the csv for easier Excel import
column_names.to_csv("./extractedData/output.txt", mode='a', index=False, header=False)

# totalFiles = 0

for filename in sorted(os.listdir(rawDataTesting)):
    file = os.path.join(rawDataTesting, filename)
    print('Current file:', filename)
    
    
    # checking if it is a file, assigning an array to all the file's keys
    if os.path.isfile(file):
        file = h5py.File(file, 'r')
        keys = np.array(list(file.keys()))
        beams = getBeams(file)        
        print('Number of beams:', beams.shape[0])

    # Column for spacecraft orientation
    orientation = file['orbit_info/sc_orient'][0]
        
    # Looping through all the beams for the granule
    # Keep in mind that any given beam may or may not have the land segments group
    for beam in beams:
        
            if '{}/land_segments'.format(beam) in file:

                latitude = file[beam + '/land_segments/latitude']
                latitude = np.array((latitude)).reshape((latitude.shape[0], 1))

                longitude = file[beam + '/land_segments/longitude']
                longitude = np.array((longitude)).reshape((longitude.shape[0], 1))

                # creating a matrix for the vertices of our rectangle
                # rectangleVertices = drawPolygons(longitude, latitude)            

                # Column for spacecraft orientation, reassigned in loop to match length of latitude records for particular beam
                sc_orient = np.full_like(longitude, orientation, dtype=int)
                # print(sc_orient[0:2])

                # Column for canopy height             
                h_canopy = file[beam + '/land_segments/canopy/h_canopy']
                h_canopy = np.array(h_canopy).reshape((h_canopy.shape[0], 1))       

                # Column for canopy height uncertainty
                h_canopy_uncertainty = file[beam + '/land_segments/canopy/h_canopy_uncertainty']
                h_canopy_uncertainty = np.array(h_canopy_uncertainty).reshape((h_canopy_uncertainty.shape[0], 1))        

                # This metric is NOT reshaped into a 1 column array, it has dimensions of ( number of transects x 18)
                canopy_h_metrics = file[beam + '/land_segments/canopy/canopy_h_metrics']
                canopy_h_metrics = np.array(canopy_h_metrics)

                # This metric is also NOT reshaped into a 1 column array, it has dimensions of ( number of transects x 5)
                h_canopy_20m = file[beam + '/land_segments/canopy/h_canopy_20m/']
                h_canopy_20m = np.array(h_canopy_20m)

                # Column for the canopy openness
                canopy_openness = file[beam + '/land_segments/canopy/canopy_openness']
                canopy_openness = np.array(canopy_openness).reshape((canopy_openness.shape[0], 1))

                # Column for the segment ID beginning
                segment_id_beg = file[beam + '/land_segments/segment_id_beg']
                segment_id_beg = np.array(segment_id_beg).reshape((segment_id_beg.shape[0], 1))
                # print('Segment ID beginning', segment_id_beg[0:2])

                # Column for the segment ID ending
                segment_id_end = file[beam + '/land_segments/segment_id_end']
                segment_id_end = np.array(segment_id_end).reshape((segment_id_end.shape[0], 1))

                # Column for the night flag
                night_flag = file[beam + '/land_segments/night_flag']
                night_flag = np.array(night_flag).reshape((night_flag.shape[0], 1))

                # Column for the specific file name, same dimensions as latitude column
                # Though this is the first column in the eventual csv, it is lower in this loop as we dont yet know # of records per specific beam. 
                # That information is obtained with the latitude flag at the start of the loop
                file_name = np.full_like(latitude, filename[:-3], dtype=np.dtype('U100'))

                # Creating a column for the granule date.
                # HMS data can be parsed later as there's inconsistencies between file names and the granule START time listed in the Earthdata portal. 
                # Creating a datetime object of the filename string, then a column of just the datetime information
                date_time = datetime.strptime(filename[filename.index('_') + 1:filename.index('_') + 15], "%Y%m%d%H%M%S")
                date = np.full_like(latitude, date_time, dtype=np.dtype('U100'))
                date = np.array(date).reshape((date.shape[0], 1))

                # Creating a column for the specific beam name, same dimensions as the latitude column
                beam_name = np.full_like(file_name, beam, dtype=np.dtype('U100'))            

                records_in_beam = latitude.shape[0]
                # print('beam',beam,'has', records_in_beam, 'records')

                overall_records = overall_records + records_in_beam
                # print('output csv will have', overall_records, 'records with addition of', beam)

                df = pd.DataFrame(np.hstack((
                        file_name,
                        date,
                        beam_name,            
                        latitude, 
                        longitude, 
                        sc_orient, 
                        h_canopy, 
                        h_canopy_uncertainty, 
                        canopy_h_metrics, 
                        h_canopy_20m,
                        canopy_openness, 
                        segment_id_beg, 
                        segment_id_end, 
                        night_flag                            
                    )))

                df.to_csv(output_file, mode='a', index=False, header=False)
    print('Success! File done.')
    totalFiles = totalFiles + 1
            
            
print('Total files read:', totalFiles)

Current file: ATL08_20190323011928_12920206_006_02.h5


OSError: Unable to open file (file signature not found)

In [ ]:
# def drawPolygons(latitude, longitude):
#     # input: array of points in their latitude & longitude form
#     # output: array of rectangles
#     # LONGITUDE = X, LATITUDE = y
    
#     rectangleVertices = np.zeros((latitude.shape[0], 4), dtype=np.double)
#     print(rectangleVertices.shape)
    
#     coordinates = np.hstack((longitude, latitude))
#     for coordinate in coordinates:
#         long = coordinate[0]
#         lat = coordinate[1]
        
        # INSERT CODE HERE TO CONVERT COORDINATES TO LOCAL UTM
        # longMeters = 
        # latMeters = 
    
        
        # topLeft = (longMeters - 4.75903, lat + 49.9695)
        # topRight = (longMeters + 8.24897, lat + 49.9695)
        # bottomRight = (longMeters + 4.75903, lat - 49.9695)
        # bottomLeft = (longMeters - 8.24897, lat - 49.9695)

- The code crashed on file: ATL08_20230117182842_04311802_006_01.h5 
- Previous files had all 6 keys, this one only had 2
- Need to accommodate varying number of keys rather than assume they all have 6
- Lets try the next file: ATL08_20230118072730_04391806_006_01.h5
- Next one has 6
- Code also crashed on ATL08_20190328005358_13680207_006_02.h5, as there's only 2 beams
- for one of the beams there is no land_segments key
- need to incorporate error handling for if there's no land segments key

In [17]:
# This file only has 2 beams
# Of the two beams, gt1r has no land_segments group

# errorFile = 'rawDataTesting/ATL08_20190328005358_13680207_006_02.h5'
# errorFile = h5py.File(errorFile, 'r')
# list(errorFile['gt1r'].keys())
# if 'gt1l/land_segments' in errorFile:
#     print('This beam does have a land_segments group')
# else:
#     print(' This beam DOES NOT have a land_segments group')
# 0getBeams(errorFile)

This beam does have a land_segments group


Code loaded about 33,000,000 lines which I loaded into an excel query 
Ran into an issue with one of the files
Divide file directory into 20 subfiles
Go 250 files, then move onto another .txt file
Use JMP to view files
Text editor for dealing with gigantic files
Column edits in a text editor
ultraEdit or sublimeText
Linux Commands for printing first few lines of text file
Texas A&M Moambo, Propescu, Amy Neuenschwander
GEDI Dubai, Armsten, Duncanson
Not all monolithic, lean toward RH98
Look into recent releases using V005 or V006 data
Try initially to get a reduced geographic area
1 dataset of what geographic area is, broken into smaller files

Spring Courses:
- Val's lidar
- Quinn Thomas's course?
- Potentially Machine learning with Anuj
- Talk to Anuj about machine learning and linear algebra background
- either biometry or stats & research (5616)

In [ ]:

# Error handling for missing keys
# Relevant stackOverflow problems
# https://stackoverflow.com/questions/7675838/python-hdf5-h5py-issues-opening-multiple-files?rq=3
# https://stackoverflow.com/questions/55473368/h5py-randomly-unable-to-open-object-component-not-found
# https://stackoverflow.com/questions/41474634/python-unable-to-open-a-h5-file
# Think there's some funny business with the back slashes in the directory assigments, double check about escapes
#
